In [ ]:
import numpy as np
import pandas as pd
import pyshark
import matplotlib.pyplot as plt
import json # original json library
import logging

In [ ]:
import sys
sys.path.append('/home/benjamin/Folders_Python/Cyber/libs')

In [ ]:
LOG_FILENAME = '/home/benjamin/Folders_Python/Cyber/logs/logfile.log'
LOG_FORMAT = '%(asctime)% -- %(name)s -- %(levelname)s -- %(message)s'
# LOG_LEVEL = logging.INFO

# specific logger for the module
logger = logging.getLogger(__name__)   # creates specific logger for the module
logger.setLevel(logging.DEBUG)    # entry level of messages from all handlers
LOG_FORMAT = '%(asctime)s -- %(name)s -- %(levelname)s -- %(message)s'
formatter = logging.Formatter(LOG_FORMAT)

# file handler to log everything
file_handler = logging.FileHandler(LOG_FILENAME, mode='w')
file_handler.setLevel(logging.DEBUG)  # all messages (DEBUG and up) get logged in the file
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

# stream handler to show messages to the console
console = logging.StreamHandler()
console.setLevel(logging.WARNING)  # Warning messages and up get displayed to the console
console.setFormatter(formatter)
logger.addHandler(console)

# start your engine
logger.info("-------- new run --------")

In [ ]:
# PCAPFILE = '/home/benjamin/Folders_Python/Cyber/data/input_pcaps/test.pcap'
PCAPFILE = '/home/benjamin/Folders_Python/Cyber/data/input_pcaps/smallFlows.pcap'
# PCAPFILE = '/home/benjamin/Folders_Python/Cyber/data/input_pcaps/test.pcap'

logger.info(f'-- entry pcap file = {PCAPFILE} --')

In [ ]:
capture = pyshark.FileCapture(
    input_file=PCAPFILE,
    use_ek=True
)

print(capture)

pkt = capture[0]

In [ ]:
dir(pkt)

In [ ]:
list_layers = pkt.layers

print(list_layers)

In [ ]:
for i,layer in enumerate(list_layers):
    fields_names = layer.field_names
    print(f'layer {i} = {fields_names}')
    for field in fields_names:
        print(f"layer {i} -- {field} = {layer.get(field)}")
    print(f'\n')

In [ ]:
capture = pyshark.FileCapture(
    input_file=PCAPFILE,
    use_ek=False
)

In [ ]:
# for i, pkt in enumerate(capture):
#     pkt.pretty_print()
#     print(f'-- {i} ----------------------------------------------------------\n')

In [ ]:
import cyberlib as cbl

In [ ]:
pkt = capture[0]

In [ ]:
paquet = cbl.PyPacket(pkt)

In [ ]:
paquet.data

In [ ]:
df = paquet.dataframe

In [ ]:
df

In [ ]:
# run the 3 pcaps into a dataframe and save them

file_dict = {
    'test.pcap' : '/home/benjamin/Folders_Python/Cyber/data/input_pcaps/test.pcap',
    'smallFlows.pcap' : '/home/benjamin/Folders_Python/Cyber/data/input_pcaps/smallFlows.pcap',
    'bigFlows.pcap' : '/home/benjamin/Folders_Python/Cyber/data/input_pcaps/bigFlows.pcap'
}

DIRPATH = '/home/benjamin/Folders_Python/Cyber/data/dataframes/'

In [ ]:
for filename, filepath in file_dict.items():
    print(f"processing {filename}")
    
    capture = pyshark.FileCapture(
        input_file=filepath,
        use_ek=False
        )
    
    df_full = pd.DataFrame()
    i=0

    while True:
        try:
            pkt = capture.next()
            paquet = cbl.PyPacket(pkt)
            df = paquet.dataframe
            df_full = pd.concat([df_full, df], axis=0)
            i += 1
            print (f'processing packet number {i}', end='\r')
        except StopIteration as e:
            logger.info(f"reached end of capture after reading {i} packets")
            break
        
    df_full = df_full.reset_index(drop=True)
    
    savename = DIRPATH + filename + '.pkl'
    df_full.to_pickle(savename)
    print('\n')
    print(f'saving {filename} as pickle')